# 🧠 DeFi Wallet Credit Scoring - Jupyter Notebook Version
This notebook processes DeFi transaction data to assign credit scores to wallet addresses based on their on-chain behavior.

### 📦 Import Required Libraries

In [ ]:
import json
import pandas as pd
from tqdm import tqdm

### 📥 Load and Clean JSON Data

In [ ]:

# Load JSON (list of transactions)
with open("user_transactions.json", "r") as f:
    data = json.load(f)

# Flatten and clean the data
cleaned = []
for tx in tqdm(data):
    try:
        cleaned.append({
            'wallet': tx['userWallet'],
            'action': tx['action'].lower(),
            'timestamp': tx['timestamp'],
            'amount': float(tx['actionData'].get('amount', 0)) / 1e6  # Assuming USDC, 6 decimals
        })
    except Exception as e:
        print("Skipping tx due to error:", e)


### 🧾 Create DataFrame

In [ ]:

df = pd.DataFrame(cleaned)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')


### 📊 Feature Extraction

In [ ]:

def extract_features(group):
    features = {}
    actions = group['action'].value_counts().to_dict()
    features['num_transactions'] = len(group)
    features['unique_actions'] = group['action'].nunique()
    features['num_deposit'] = actions.get('deposit', 0)
    features['num_borrow'] = actions.get('borrow', 0)
    features['num_repay'] = actions.get('repay', 0)
    features['num_liquidation'] = actions.get('liquidationcall', 0)
    features['total_deposit'] = group[group['action'] == 'deposit']['amount'].sum()
    features['total_borrow'] = group[group['action'] == 'borrow']['amount'].sum()
    features['total_repay'] = group[group['action'] == 'repay']['amount'].sum()
    features['deposit_to_borrow'] = (features['total_deposit'] / features['total_borrow']
                                     if features['total_borrow'] > 0 else 0)
    features['repay_to_borrow'] = (features['total_repay'] / features['total_borrow']
                                   if features['total_borrow'] > 0 else 0)
    features['active_days'] = group['timestamp'].dt.date.nunique()
    features['lifetime_days'] = (group['timestamp'].max() - group['timestamp'].min()).days + 1
    return pd.Series(features)


### 👥 Group by Wallet and Apply Feature Extraction

In [ ]:

wallet_features = df.groupby('wallet', group_keys=False).apply(extract_features).fillna(0)


### 🧠 Credit Scoring Logic

In [ ]:

def score_wallet(row):
    score = 600
    score += min(row['deposit_to_borrow'] * 50, 100)
    score += min(row['repay_to_borrow'] * 100, 100)
    score += min(row['active_days'] * 2, 100)
    score -= min(row['num_liquidation'] * 50, 200)
    return score

wallet_features['credit_score'] = wallet_features.apply(score_wallet, axis=1)


### 💾 Save Credit Scores to CSV

In [ ]:

wallet_features[['credit_score']].to_csv("wallet_scores.csv", index=True)
print("✅ File saved as wallet_scores.csv")
